<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [ ]:
!rm -rf logs

In [9]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 21s 795ms/step - loss: 0.6782 - accuracy: 0.5706 - val_loss: 0.6685 - val_accuracy: 0.6683
Epoch 2/10
26/26 [==============================] - 20s 765ms/step - loss: 0.6424 - accuracy: 0.7165 - val_loss: 0.6409 - val_accuracy: 0.6634
Epoch 3/10
26/26 [==============================] - 19s 750ms/step - loss: 0.6000 - accuracy: 0.7214 - val_loss: 0.5977 - val_accuracy: 0.7220
Epoch 4/10
26/26 [==============================] - 19s 746ms/step - loss: 0.5414 - accuracy: 0.7530 - val_loss: 0.5099 - val_accuracy: 0.7561
Epoch 5/10
26/26 [==============================] - 19s 750ms/step - loss: 0.4807 - accuracy: 0.7835 - val_loss: 0.4741 - val_accuracy: 0.7951
Epoch 6/10
26/26 [==============================] - 19s 746ms/step - loss: 0.4261 - accuracy: 0.8285 - val_loss: 0.3896 - val_accuracy: 0.8585
Epoch 7/10
26/26 [==============================] - 20s 754ms/step - loss: 0.3776 - accuracy: 0.8552 - val_loss: 0.3525 - val_accuracy: 0.8488

In [10]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [11]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.65.h5
26/26 - 20s - loss: 0.6755 - accuracy: 0.5633 - val_loss: 0.6522 - val_accuracy: 0.8195
Epoch 2/5

Epoch 00002: saving model to weights.02-0.61.h5
26/26 - 20s - loss: 0.6190 - accuracy: 0.7336 - val_loss: 0.6052 - val_accuracy: 0.7122
Epoch 3/5

Epoch 00003: saving model to weights.03-0.70.h5
26/26 - 19s - loss: 0.5635 - accuracy: 0.7530 - val_loss: 0.7009 - val_accuracy: 0.4732
Epoch 4/5

Epoch 00004: saving model to weights.04-0.53.h5
26/26 - 20s - loss: 0.5105 - accuracy: 0.7628 - val_loss: 0.5257 - val_accuracy: 0.7512
Epoch 5/5

Epoch 00005: saving model to weights.05-0.43.h5
26/26 - 19s - loss: 0.4631 - accuracy: 0.7920 - val_loss: 0.4345 - val_accuracy: 0.8293


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 21s - loss: 0.6608 - accuracy: 0.6180 - val_loss: 0.6387 - val_accuracy: 0.7854


In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 21s - loss: 0.6742 - accuracy: 0.5633 - val_loss: 0.6579 - val_accuracy: 0.7317
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 20s - loss: 0.6373 - accuracy: 0.6910 - val_loss: 0.6437 - val_accuracy: 0.5805


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 20s - loss: 0.6720 - accuracy: 0.5852 - val_loss: 0.6350 - val_accuracy: 0.8341
Epoch 2/50
26/26 - 22s - loss: 0.6079 - accuracy: 0.7433 - val_loss: 0.6484 - val_accuracy: 0.5317
Epoch 3/50
26/26 - 20s - loss: 0.5329 - accuracy: 0.7628 - val_loss: 0.4893 - val_accuracy: 0.8244
Epoch 4/50
26/26 - 20s - loss: 0.4752 - accuracy: 0.8175 - val_loss: 0.5662 - val_accuracy: 0.6634
Epoch 5/50
26/26 - 20s - loss: 0.4039 - accuracy: 0.8467 - val_loss: 0.3534 - val_accuracy: 0.8780
Epoch 6/50
26/26 - 20s - loss: 0.3342 - accuracy: 0.8844 - val_loss: 0.4003 - val_accuracy: 0.8244
Epoch 7/50
26/26 - 20s - loss: 0.2691 - accuracy: 0.9027 - val_loss: 0.4249 - val_accuracy: 0.7854
Epoch 8/50
26/26 - 20s - loss: 0.2464 - accuracy: 0.9148 - val_loss: 0.1717 - val_accuracy: 0.9756
Epoch 9/50
26/26 - 20s - loss: 0.1720 - accuracy: 0.9574 - val_loss: 0.3181 - val_accuracy: 0.8488
Epoch 10/50
26/26 - 21s - loss: 0.1523 - accuracy: 0.9611 - val_loss: 0.1147 - val_accuracy: 0.9854
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [17]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 21s 799ms/step - loss: 0.6738 - accuracy: 0.5876 - val_loss: 0.6474 - val_accuracy: 0.7610
Epoch 2/5
26/26 [==============================] - 22s 847ms/step - loss: 0.6253 - accuracy: 0.6934 - val_loss: 0.6066 - val_accuracy: 0.7463
Epoch 3/5
26/26 [==============================] - 20s 779ms/step - loss: 0.5724 - accuracy: 0.7372 - val_loss: 0.5664 - val_accuracy: 0.7463
Epoch 4/5
26/26 [==============================] - 20s 781ms/step - loss: 0.5001 - accuracy: 0.7810 - val_loss: 0.4727 - val_accuracy: 0.7756
Epoch 5/5
26/26 [==============================] - 20s 770ms/step - loss: 0.4556 - accuracy: 0.7993 - val_loss: 0.5156 - val_accuracy: 0.7220


In [34]:
pd.read_csv(csv_file)

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.587591,0.674217,0.760976,0.647391
1,1,0.693431,0.625522,0.746341,0.606644
2,2,0.737226,0.573194,0.746341,0.566379
3,3,0.781022,0.500925,0.775610,0.472661
4,4,0.799270,0.457036,0.721951,0.515556


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [35]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr


model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 23s 891ms/step - loss: 0.6823 - accuracy: 0.5328 - val_loss: 0.6836 - val_accuracy: 0.4634

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 21s 799ms/step - loss: 0.6629 - accuracy: 0.5693 - val_loss: 0.6792 - val_accuracy: 0.4634

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 20s 773ms/step - loss: 0.6541 - accuracy: 0.5779 - val_loss: 0.6698 - val_accuracy: 0.5024

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 21s 799ms/step - loss: 0.6491 - accuracy: 0.5900 - val_loss: 0.6665 - val_accuracy: 0.5171

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 20s 785ms/step - loss: 0.6

In [36]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [37]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])            

Epoch 1/50
26/26 [==============================] - 30s 1s/step - loss: 0.6695 - accuracy: 0.6071 - val_loss: 0.6376 - val_accuracy: 0.6976
Epoch 2/50
26/26 [==============================] - 21s 811ms/step - loss: 0.6210 - accuracy: 0.6971 - val_loss: 0.5867 - val_accuracy: 0.8293
Epoch 3/50
26/26 [==============================] - 22s 858ms/step - loss: 0.5841 - accuracy: 0.7202 - val_loss: 0.5200 - val_accuracy: 0.8537
Epoch 4/50
26/26 [==============================] - 21s 816ms/step - loss: 0.4936 - accuracy: 0.8297 - val_loss: 0.4256 - val_accuracy: 0.8537
Epoch 5/50
26/26 [==============================] - 21s 816ms/step - loss: 0.4446 - accuracy: 0.8054 - val_loss: 0.4027 - val_accuracy: 0.8683
Epoch 6/50
26/26 [==============================] - 25s 944ms/step - loss: 0.3595 - accuracy: 0.8710 - val_loss: 0.3385 - val_accuracy: 0.8976
Epoch 7/50
26/26 [==============================] - 21s 789ms/step - loss: 0.2931 - accuracy: 0.9039 - val_loss: 0.2252 - val_accuracy: 0.9366
Ep

In [ ]:
%tensorboard --logdir log_dir